## Step 1: Scrape Tweet Data

In [1]:
# import GetOldTweets3 as got
import tweepy
import pandas as pd
import warnings
import re
import numpy as np

# NTLK functions
import nltk
from nltk.corpus import stopwords
from nltk import tokenize as tok
from nltk.stem.snowball import SnowballStemmer # load nltk's SnowballStemmer as variabled 'stemmer'
# import lda # topic modeling -NMF & LDA
import string
from nltk.tag import StanfordNERTagger

warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
# Tf-Idf and Clustering packages
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [2]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAFYukQEAAAAApZQpuKapMCSBFMHu%2Ba1bySvK2EM%3DJNec4foagBf1eRvl240UJxO8SnkXL6mwWkQXr80HKxA1JBCkoy'
client = tweepy.Client(bearer_token = bearer_token, wait_on_rate_limit=True)

In [17]:
search_terms = ['mortgage','current account','savings account','insurance','credit card','pension',
                'personal loan','money transfer','tax advice','investment','wealth management']
# search_terms = ['trump', 'gop', 'biden', 'democrats']

In [31]:
max_results = 100
expansions = ['author_id']
user_fields = ['description','id','name','username']
tweet_fields=['id', 'author_id', 'text', 'lang', 'public_metrics', 'created_at', 'entities']
complete_data = {}
for term in search_terms:
    complete_data[term] = {}
    fields = 0
    print(term)
    query = f'{term} lang:en -is:retweet -has:media'
    for i,tweet_batch in  enumerate(tweepy.Paginator(client.search_recent_tweets, tweet_fields=tweet_fields,query=query, max_results=max_results, limit=100)):
        complete_data[term][fields] = tweet_batch
        fields+=1

mortgage
current account
savings account
insurance
credit card
pension


Rate limit exceeded. Sleeping for 507 seconds.


personal loan
money transfer
tax advice
investment
wealth management


In [33]:
complete_data['current account'][0].data[4].entities

{'annotations': [{'start': 50,
   'end': 66,
   'probability': 0.5435,
   'type': 'Other',
   'normalized_text': 'kapila pashu ahar'}],
 'mentions': [{'start': 0,
   'end': 16,
   'username': 'AbhasHalakhandi',
   'id': '886946656423337985'}]}

In [79]:
# tweet_df = pd.DataFrame(columns=['id','author_id','text','date', 'lang', 'search_term'])
list_of_tweets = []
for term in complete_data:
    for i in complete_data[term]:

        for tweet in complete_data[term][i].data:
            id= tweet.id
            author_id = tweet.author_id
            text = tweet.text
            date = tweet.created_at
            lang = tweet.lang
            # temp_df = pd.DataFrame({'id':[id], 'author_id':[author_id],'text':text, 'date':date, "lang":lang, 'search_term':term })
            # tweet_df = tweet_df.append(temp_df)
            temp = {'id':id, 'author_id':author_id,'text':text, 'date':date, "lang":lang, 'search_term':term }
            list_of_tweets.append(temp)


In [80]:
tweet_df = pd.DataFrame(list_of_tweets)

In [3]:
tweet_df = pd.read_csv("./data_with_search_term.csv")

In [4]:
# tweet_df = pd.read_csv("collected_data.csv")
tweet_df_copy = tweet_df
tweet_df

,Unnamed: 0,id,author_id,text,date,lang,search_term
0,0,1602612821585694721,1580423829801009152,@AltMediaWatch @SmilyngAssassin @Tugboat882191...,2022-12-13 10:34:06+00:00,en,mortgage
1,1,1602612794360307714,1425092150975401996,@RalphNeale1 @CTVNews Trudeau had *nothing* to...,2022-12-13 10:33:59+00:00,en,mortgage
2,2,1602612758645805056,1556214967573118980,BREAKING Jag Singer threatens to give up pensi...,2022-12-13 10:33:51+00:00,en,mortgage
3,3,1602612676168798209,703144956,@wetthroatbabyy @nojumper Hes been paying a mo...,2022-12-13 10:33:31+00:00,en,mortgage
4,4,1602612625904394240,1425092150975401996,@khumphries1 @CTVNews @JustinTrudeau Trudeau h...,2022-12-13 10:33:19+00:00,en,mortgage
...,...,...,...,...,...,...,...
69838,69838,1600090654167687168,83786013,Job Japan Private Banker Wealth Management. 35...,2022-12-06 11:31:54+00:00,en,wealth management
69839,69839,1600089915387219968,3290091422,From Wealth Management Professional to Career ...,2022-12-06 11:28:58+00:00,en,wealth management
69840,69840,1600086634665431041,1587324685414322177,@WakeOfWeek @JayRocha82 @RobSchneider @Yankees...,2022-12-06 11:15:56+00:00,en,wealth management
69841,69841,1600081033373827073,993891818998771719,BMO Wealth Management Names NewLeadership http...,2022-12-06 10:53:41+00:00,en,wealth management


In [5]:
for _, tweet in tweet_df.iterrows(): 
    tweet_df['text'][_] = tweet.text.encode('ascii', 'ignore').decode('ascii') 

C:\Users\ameym\AppData\Local\Temp\ipykernel_10008\2030783118.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_df['text'][_] = tweet.text.encode('ascii', 'ignore').decode('ascii')


In [6]:
tweet_df

,Unnamed: 0,id,author_id,text,date,lang,search_term
0,0,1602612821585694721,1580423829801009152,@AltMediaWatch @SmilyngAssassin @Tugboat882191...,2022-12-13 10:34:06+00:00,en,mortgage
1,1,1602612794360307714,1425092150975401996,@RalphNeale1 @CTVNews Trudeau had *nothing* to...,2022-12-13 10:33:59+00:00,en,mortgage
2,2,1602612758645805056,1556214967573118980,BREAKING Jag Singer threatens to give up pensi...,2022-12-13 10:33:51+00:00,en,mortgage
3,3,1602612676168798209,703144956,@wetthroatbabyy @nojumper Hes been paying a mo...,2022-12-13 10:33:31+00:00,en,mortgage
4,4,1602612625904394240,1425092150975401996,@khumphries1 @CTVNews @JustinTrudeau Trudeau h...,2022-12-13 10:33:19+00:00,en,mortgage
...,...,...,...,...,...,...,...
69838,69838,1600090654167687168,83786013,Job Japan Private Banker Wealth Management. 35...,2022-12-06 11:31:54+00:00,en,wealth management
69839,69839,1600089915387219968,3290091422,From Wealth Management Professional to Career ...,2022-12-06 11:28:58+00:00,en,wealth management
69840,69840,1600086634665431041,1587324685414322177,@WakeOfWeek @JayRocha82 @RobSchneider @Yankees...,2022-12-06 11:15:56+00:00,en,wealth management
69841,69841,1600081033373827073,993891818998771719,BMO Wealth Management Names NewLeadership http...,2022-12-06 10:53:41+00:00,en,wealth management


In [7]:
tweet_df.to_csv("data_with_search_term.csv")

In [8]:
tweet_df_all = tweet_df
tweet_df_all = tweet_df_all[tweet_df_all['text'].notna()]

In [9]:
print(tweet_df_all.shape);tweet_df_all.head()

(69843, 7)


,Unnamed: 0,id,author_id,text,date,lang,search_term
0,0,1602612821585694721,1580423829801009152,@AltMediaWatch @SmilyngAssassin @Tugboat882191...,2022-12-13 10:34:06+00:00,en,mortgage
1,1,1602612794360307714,1425092150975401996,@RalphNeale1 @CTVNews Trudeau had *nothing* to...,2022-12-13 10:33:59+00:00,en,mortgage
2,2,1602612758645805056,1556214967573118980,BREAKING Jag Singer threatens to give up pensi...,2022-12-13 10:33:51+00:00,en,mortgage
3,3,1602612676168798209,703144956,@wetthroatbabyy @nojumper Hes been paying a mo...,2022-12-13 10:33:31+00:00,en,mortgage
4,4,1602612625904394240,1425092150975401996,@khumphries1 @CTVNews @JustinTrudeau Trudeau h...,2022-12-13 10:33:19+00:00,en,mortgage


In [10]:
tweet_df_all['text']

0        @AltMediaWatch @SmilyngAssassin @Tugboat882191...
1        @RalphNeale1 @CTVNews Trudeau had *nothing* to...
2        BREAKING Jag Singer threatens to give up pensi...
3        @wetthroatbabyy @nojumper Hes been paying a mo...
4        @khumphries1 @CTVNews @JustinTrudeau Trudeau h...
                               ...                        
69838    Job Japan Private Banker Wealth Management. 35...
69839    From Wealth Management Professional to Career ...
69840    @WakeOfWeek @JayRocha82 @RobSchneider @Yankees...
69841    BMO Wealth Management Names NewLeadership http...
69842    BMO Wealth Management Names NewLeadership http...
Name: text, Length: 69843, dtype: object

In [11]:
tweet_df_all

,Unnamed: 0,id,author_id,text,date,lang,search_term
0,0,1602612821585694721,1580423829801009152,@AltMediaWatch @SmilyngAssassin @Tugboat882191...,2022-12-13 10:34:06+00:00,en,mortgage
1,1,1602612794360307714,1425092150975401996,@RalphNeale1 @CTVNews Trudeau had *nothing* to...,2022-12-13 10:33:59+00:00,en,mortgage
2,2,1602612758645805056,1556214967573118980,BREAKING Jag Singer threatens to give up pensi...,2022-12-13 10:33:51+00:00,en,mortgage
3,3,1602612676168798209,703144956,@wetthroatbabyy @nojumper Hes been paying a mo...,2022-12-13 10:33:31+00:00,en,mortgage
4,4,1602612625904394240,1425092150975401996,@khumphries1 @CTVNews @JustinTrudeau Trudeau h...,2022-12-13 10:33:19+00:00,en,mortgage
...,...,...,...,...,...,...,...
69838,69838,1600090654167687168,83786013,Job Japan Private Banker Wealth Management. 35...,2022-12-06 11:31:54+00:00,en,wealth management
69839,69839,1600089915387219968,3290091422,From Wealth Management Professional to Career ...,2022-12-06 11:28:58+00:00,en,wealth management
69840,69840,1600086634665431041,1587324685414322177,@WakeOfWeek @JayRocha82 @RobSchneider @Yankees...,2022-12-06 11:15:56+00:00,en,wealth management
69841,69841,1600081033373827073,993891818998771719,BMO Wealth Management Names NewLeadership http...,2022-12-06 10:53:41+00:00,en,wealth management


In [12]:
tweet_df_comp = tweet_df_all

In [13]:
tweet_df_comp.groupby('search_term')['id'].count()

search_term
credit card          9817
current account      3686
insurance            9872
investment           9987
money transfer       7246
mortgage             9964
pension              9972
personal loan        1440
savings account      4535
tax advice            641
wealth management    2683
Name: id, dtype: int64

## Step 3: Topic Extraction with LDA model

In [14]:
#remove unnessary words
#Complie all regular expressions
isURL = re.compile(r'http[s]?:// (?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', re.VERBOSE | re.IGNORECASE)
isRTusername = re.compile(r'^RT+[\s]+(@[\w_]+:)',re.VERBOSE | re.IGNORECASE) #r'^RT+[\s]+(@[\w_]+:)'
isEntity = re.compile(r'@[\w_]+', re.VERBOSE | re.IGNORECASE)

# Helper functions
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])) 

# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
     
        
def clean_tweet(row):
    row = isURL.sub("",row)
    row = isRTusername.sub("",row)
    row = isEntity.sub("",row)
    row = re.sub('[#|\n|\t]+', '', row)
    return row

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in tok.sent_tokenize(text) for word in tok.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens


In [15]:
# remove urls and retweets and entities from the text
tweet_df_comp['text_clean'] = tweet_df_comp['text'].apply(lambda row:clean_tweet(row))

#remove punctuations
RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
tweet_df_comp['text_clean'] = tweet_df_comp['text_clean'].str.replace(RE_PUNCTUATION, "")
tweet_df_comp.head()

C:\Users\ameym\AppData\Local\Temp\ipykernel_10008\1323273592.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet_df_comp['text_clean'] = tweet_df_comp['text_clean'].str.replace(RE_PUNCTUATION, "")


,Unnamed: 0,id,author_id,text,date,lang,search_term,text_clean
0,0,1602612821585694721,1580423829801009152,@AltMediaWatch @SmilyngAssassin @Tugboat882191...,2022-12-13 10:34:06+00:00,en,mortgage,No record and a juicy false accusation cla...
1,1,1602612794360307714,1425092150975401996,@RalphNeale1 @CTVNews Trudeau had *nothing* to...,2022-12-13 10:33:59+00:00,en,mortgage,Trudeau had nothing to do with mortgage rate...
2,2,1602612758645805056,1556214967573118980,BREAKING Jag Singer threatens to give up pensi...,2022-12-13 10:33:51+00:00,en,mortgage,BREAKING Jag Singer threatens to give up pensi...
3,3,1602612676168798209,703144956,@wetthroatbabyy @nojumper Hes been paying a mo...,2022-12-13 10:33:31+00:00,en,mortgage,Hes been paying a mortgage for 12 years
4,4,1602612625904394240,1425092150975401996,@khumphries1 @CTVNews @JustinTrudeau Trudeau h...,2022-12-13 10:33:19+00:00,en,mortgage,Trudeau had nothing to do with mortgage rat...


In [18]:
# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("./twitter-analytics/pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')
readInStopwords.extend(search_terms)

stop_list = stop_words + readInStopwords
stop_list = list(set(stop_list)) 

In [19]:
# parameter for lda, i am selecrign 3 topic and 4 words for each of the search terms 
number_topics = 5
number_words = 5

In [20]:
tweet_df_comp['search_term'].unique()

array(['mortgage', 'current account', 'savings account', 'insurance',
       'credit card', 'pension', 'personal loan', 'money transfer',
       'tax advice', 'investment', 'wealth management'], dtype=object)

In [21]:
tweets_all_topics= pd.DataFrame()
# term frequency modelling
for terms in tweet_df_comp['search_term'].unique():
    print(terms)
    tweets_search_topics  = tweet_df_comp[tweet_df_comp['search_term']==terms].reset_index(drop=True)
    corpus = tweets_search_topics['text_clean'].tolist()
    # print(corpus)
    tf_vectorizer = CountVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
    tf = tf_vectorizer.fit_transform(corpus)
    
    # Create and fit the LDA model
    model = LDA(n_components=number_topics, n_jobs=-1)
    id_topic = model.fit(tf)
    # Print the topics found by the LDA model
    print("Topics found via LDA:")
    topic_keywords = show_topics(vectorizer=tf_vectorizer, lda_model=model, n_words=number_words)        
    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    df_topic_keywords = df_topic_keywords.reset_index()
    df_topic_keywords['topic_index'] = df_topic_keywords['index'].str.split(' ', n = 1, expand = True)[[1]].astype('int')
    print(df_topic_keywords)
    
    ############ get the dominat topic for each document in a data frame ###############
    # Create Document — Topic Matrix
    lda_output = model.transform(tf)
    # column names
    topicnames = ["Topic" + str(i) for i in range(model.n_components)]
    # index names
    docnames = ["Doc" + str(i) for i in range(len(corpus))]
    
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic   
    df_document_topic = df_document_topic.reset_index()
        
    #combine all the search terms into one data frame
    tweets_topics = tweets_search_topics.merge(df_document_topic, left_index=True, right_index=True, how='left')
    tweets_topics_words = tweets_topics.merge(df_topic_keywords, how='left', left_on='dominant_topic', right_on='topic_index')
    tweets_all_topics = tweets_all_topics.append(tweets_topics_words)

mortgage


c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'able', 'account', 'across', 'advice', 'ai', 'almost', 'also', 'among', 'ca', 'card', 'credit', 'current', 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', 'however', 'least', 'let', 'like', 'likely', 'loan', 'management', 'may', 'might', 'money', 'must', "n't", 'need', 'neither', 'often', 'personal', 'rather', 'said', 'savings', 'say', 'says', 'sha', 'since', 'tax', 'tis', 'transfer', 'twas', 'us', 'wants', 'wealth', 'wo', 'yet'] not in stop_words.
  warnings.warn(


Topics found via LDA:
     index    Word 0     Word 1     Word 2   Word 3      Word 4  topic_index
0  Topic 0     deals        amp         us      see       rates            0
1  Topic 1       pay      rates       home     like       house            1
2  Topic 2       pay       rent      rates      get      people            2
3  Topic 3     years      rates  available  maximum     checked            3
4  Topic 4  received  responses      years   latest  homebuyers            4


c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\ameym\AppData\Local\Temp\ipykernel_10008\496991535.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets_all_topics = tweets_all_topics.append(tweets_topics_words)
c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'able', 'account', 'across', 'advice', 'ai', 'almost', 'also', 'among', 'ca', 'card', 'credit', 'current', 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', 'however', 'least

current account
Topics found via LDA:
     index   Word 0   Word 1   Word 2     Word 3    Word 4  topic_index
0  Topic 0    start    shows     prev  investing     chart            0
1  Topic 1      buy     take    build  community  benefits            1
2  Topic 2       us   please     know       help        hi            2
3  Topic 3     bank  deficit  billion        one     still            3
4  Topic 4  twitter   people      one         im      dont            4


...

c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\ameym\AppData\Local\Temp\ipykernel_10008\496991535.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets_all_topics = tweets_all_topics.append(tweets_topics_words)
c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'able', 'account', 'across', 'advice', 'ai', 'almost', 'also', 'among', 'ca', 'card', 'credit', 'current', 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', 'however', 'least

Topics found via LDA:
     index       Word 0   Word 1       Word 2  Word 3   Word 4  topic_index
0  Topic 0       crypto  program       little   group     join            0
1  Topic 1    following       md  instruction      x3  telegrm            1
2  Topic 2        money      new         dont     amp     like            2
3  Topic 3         link    click         capo    free   little            3
4  Topic 4  opportunity     come        worth  crypto    world            4


c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\ameym\AppData\Local\Temp\ipykernel_10008\496991535.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets_all_topics = tweets_all_topics.append(tweets_topics_words)
c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'able', 'account', 'across', 'advice', 'ai', 'almost', 'also', 'among', 'ca', 'card', 'credit', 'current', 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', 'however', 'least

wealth management
Topics found via LDA:
     index  Word 0      Word 1 Word 2     Word 3     Word 4  topic_index
0  Topic 0  wealth  management     us      banks  announced            0
1  Topic 1    read    economic     us     stocks    remains            1
2  Topic 2  wealth  management   help  financial    article            2
3  Topic 3  report        read  ahead         us       move            3
4  Topic 4  wealth  management    one      offer    leading            4


c:\Users\ameym\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\ameym\AppData\Local\Temp\ipykernel_10008\496991535.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets_all_topics = tweets_all_topics.append(tweets_topics_words)


In [22]:
tweets_all_topics = tweets_all_topics.reset_index(drop=True)
print(tweets_all_topics.shape)
tweets_all_topics.head()

(69843, 22)


,Unnamed: 0,id,author_id,text,date,lang,search_term,text_clean,index_x,Topic0,...,Topic3,Topic4,dominant_topic,index_y,Word 0,Word 1,Word 2,Word 3,Word 4,topic_index
0,0,1602612821585694721,1580423829801009152,@AltMediaWatch @SmilyngAssassin @Tugboat882191...,2022-12-13 10:34:06+00:00,en,mortgage,No record and a juicy false accusation cla...,Doc0,0.26,...,0.02,0.02,1,Topic 1,pay,rates,home,like,house,1
1,1,1602612794360307714,1425092150975401996,@RalphNeale1 @CTVNews Trudeau had *nothing* to...,2022-12-13 10:33:59+00:00,en,mortgage,Trudeau had nothing to do with mortgage rate...,Doc1,0.03,...,0.27,0.03,1,Topic 1,pay,rates,home,like,house,1
2,2,1602612758645805056,1556214967573118980,BREAKING Jag Singer threatens to give up pensi...,2022-12-13 10:33:51+00:00,en,mortgage,BREAKING Jag Singer threatens to give up pensi...,Doc2,0.01,...,0.01,0.01,1,Topic 1,pay,rates,home,like,house,1
3,3,1602612676168798209,703144956,@wetthroatbabyy @nojumper Hes been paying a mo...,2022-12-13 10:33:31+00:00,en,mortgage,Hes been paying a mortgage for 12 years,Doc3,0.05,...,0.30,0.05,1,Topic 1,pay,rates,home,like,house,1
4,4,1602612625904394240,1425092150975401996,@khumphries1 @CTVNews @JustinTrudeau Trudeau h...,2022-12-13 10:33:19+00:00,en,mortgage,Trudeau had nothing to do with mortgage rat...,Doc4,0.03,...,0.53,0.03,3,Topic 3,years,rates,available,maximum,checked,3


In [23]:
tweets_all_topics.to_csv('./processed_data/tweets_all_topics.csv', index=False)

In [24]:
tweets_all_topics = pd.read_csv('./processed_data/tweets_all_topics.csv')

## Step 4: Sentiment analysis using Deep Learning

In [25]:
import keras
import h5py
from keras.models import model_from_json
from keras.models import load_model
import json
from nltk.tokenize import RegexpTokenizer


In [26]:
# read in the weight of the trained model.
weight_path = './twitter-analytics//models/dl_sentiment_model/best_weight_glove_bi_512.hdf5'
prd_model = load_model(weight_path)
prd_model.summary()
word_idx = json.load(open("./twitter-analytics//models/dl_sentiment_model/word_idx.txt"))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 56, 300)           120000300 
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dense_20 (Dense)            (None, 512)               131584    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 120,576,310
Trainable params: 576,010
Non-trainable params: 120,000,300
____________________________________

In [27]:
def get_sentiment_DL(prd_model, text_data, word_idx):

    #data = "Pass the salt"

    live_list = []
    batchSize = len(text_data)
    live_list_np = np.zeros((56,batchSize))
    for index, row in text_data.iterrows():
        #print (index)
        text_data_sample = text_data['text_clean'][index]
        # split the sentence into its words and remove any punctuations.
        tokenizer = RegexpTokenizer(r'\w+')
        text_data_list = tokenizer.tokenize(text_data_sample)

        #text_data_list = text_data_sample.split()


        labels = np.array(['1','2','3','4','5','6','7','8','9','10'], dtype = "int")
        #word_idx['I']
        # get index for the live stage
        data_index = np.array([word_idx[word.lower()] if word.lower() in word_idx else 0 for word in text_data_list])
        data_index_np = np.array(data_index)

        # padded with zeros of length 56 i.e maximum length
        padded_array = np.zeros(56)
        padded_array[:data_index_np.shape[0]] = data_index_np[:56]
        data_index_np_pad = padded_array.astype(int)


        live_list.append(data_index_np_pad)

    live_list_np = np.asarray(live_list)
    score = prd_model.predict(live_list_np, batch_size=batchSize, verbose=0)
    single_score = np.round(np.dot(score, labels)/10,decimals=2)

    score_all  = []
    for each_score in score:

        top_3_index = np.argsort(each_score)[-3:]
        top_3_scores = each_score[top_3_index]
        top_3_weights = top_3_scores/np.sum(top_3_scores)
        single_score_dot = np.round(np.dot(top_3_index, top_3_weights)/10, decimals = 2)
        score_all.append(single_score_dot)

    text_data['Sentiment_Score'] = pd.DataFrame(score_all)

    return text_data


In [28]:
text_data =  tweets_all_topics
text_out = get_sentiment_DL(prd_model, text_data, word_idx)


### example of negative tweets


In [29]:
text_out.sort_values(by='Sentiment_Score')[['text','Sentiment_Score']].head()

,text,Sentiment_Score
9152,@stinky_linke @RepsMasi @elonmusk 2) then I re...,0.03
32546,@AmericanAir the worst airline ever! Im so dis...,0.03
46035,@SaulStaniforth And again @wesstreeting showin...,0.03
16480,THEY FUCKING FORGOT TO PUT MY VACATION ON MY P...,0.03
38686,@UKurbanite @ClarkeMicah Absolute rubbish. It ...,0.03


### example of positive tweets

In [30]:
text_out.sort_values(by='Sentiment_Score', ascending=False)[['text','Sentiment_Score']].head().T

,20000,31755,58554,30168,64404
text,@ElieNYC Fabulous moments with the children. T...,I didnt have to throw any Xmas gifts on a cred...,@indiainpixels Vizag is a beautiful place with...,@Cameron_Steelh @nelo_musk @Gtwy @gregggonsalv...,@SAPoliceService Greatest desire of earning wi...
Sentiment_Score,0.85,0.85,0.84,0.83,0.82


In [31]:
text_out.query('0<= Sentiment_Score < 0.5').sort_values(by='Sentiment_Score', ascending=False).size

1317210

In [32]:
text_out.query('0.5<= Sentiment_Score').sort_values(by='Sentiment_Score', ascending=False).size

289179

In [33]:
# save the output files
text_out.to_csv('./processed_data/tweets_topics_sentiment.csv', index=False)

In [34]:
text_out = pd.read_csv("./processed_data/tweets_topics_sentiment.csv")

## Step 5:  Named Entity Recognition

The below section is implementing a stanford 3 class NER tagger. The model is trained based on on supervised Conditional Random Field (CRF) model. Additional information on the model is available at https://nlp.stanford.edu/software/CRF-NER.html

In [35]:
def get_NER(text_data):
    stanford_classifier = './twitter-analytics//models/ner/english.all.3class.distsim.crf.ser.gz'
    stanford_ner_path = './twitter-analytics//models/ner/stanford-ner.jar'

    st = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')
    print ('start get_NER')
    text_out = text_data.copy()
    doc = [ docs + ' 12345678 ' for docs in list(text_data['text'])]
    # ------------------------- Stanford Named Entity Recognition
    tokens = nltk.word_tokenize(str(doc))
    entities = st.tag(tokens) # actual tagging takes place using Stanford NER algorithm


    entities = [list(elem) for elem in entities] # Convert list of tuples to list of list
    print ('tag complete')
    for idx,element in enumerate(entities):
        try:
            if entities[idx][0] == '12345678':
                entities[idx][1] = "DOC_NUMBER"  #  Modify data by adding the tag "Doc_Number"
            elif entities[idx][1] == "PERSON" and entities[idx + 1][1] == "PERSON":
                entities[idx + 1][0] = entities[idx][0] + '-' + entities[idx+1][0]
                entities[idx][1] = 'Combined'
            # Combine consecutive Organization names
            elif entities[idx][1] == 'ORGANIZATION' and entities[idx + 1][1] == 'ORGANIZATION':
                entities[idx + 1][0] = entities[idx][0] + '-' + entities[idx+1][0]
                entities[idx][1] = 'Combined'
        except IndexError:
            break
    print ('enumerate complete')
    # Filter list of list for the words we are interested in
    filter_list = ['DOC_NUMBER','PERSON','LOCATION','ORGANIZATION']
    entityWordList = [element for element in entities if any(i in element for i in filter_list)]

    entityString = ' '.join(str(word) for insideList in entityWordList for word in insideList) # convert list to string and concatenate it
    entitySubString = entityString.split("DOC_NUMBER") # split the string using the separator 'TWEET_NUMBER'
    del entitySubString[-1] # delete the extra blank row created in the previous step

    # Store the classified NERs in the main tweet data frame
    for idx,docNER in enumerate(entitySubString):
        docNER = docNER.strip().split() # split the string into word list
        # Filter for words tagged as Organization and store it in data frame
        text_out.loc[idx,'Organization'] =  ','.join([docNER[i-1]  for i,x in enumerate(docNER) if x == 'ORGANIZATION'])
        # Filter for words tagged as LOCATION and store it in data frame
        text_out.loc[idx,'Place'] = ','.join([docNER[i-1] for i,x in enumerate(docNER) if x == 'LOCATION'])
        # Filter for words tagged as PERSON and store it in data frame
        text_out.loc[idx,'Person'] = ','.join([docNER[i-1]  for i,x in enumerate(docNER) if x == 'PERSON'])

    print ('process complete')
    return text_out

In [36]:
text_out.size

1606389

In [37]:
import math

In [38]:
text_ner_out = {}
iters =  100
for i in range(iters):
  text_ner_out[i] = get_NER(text_out.iloc[i*math.ceil(text_out.size/iters):(i+1)*math.ceil(text_out.size/iters),:])

start get_NER
tag complete
enumerate complete
process complete
start get_NER
tag complete
enumerate complete
process complete
start get_NER
tag complete
enumerate complete
process complete
start get_NER
tag complete
enumerate complete


KeyboardInterrupt: 

In [ ]:
text_ner_out_full = pd.DataFrame(text_ner_out[0])
for i in range(1,iters):
  pd.concat([text_ner_out_full, text_ner_out[i]], ignore_index=True)

In [ ]:
#the outputs of the ner tagger
text_ner_out_full.loc[(text_ner_out_full['Place'] != '') | (text_ner_out_full['Organization'] != '')|(text_ner_out_full['Person'] != '')][['text','Organization','Place','Person']].head()


,text,Organization,Place,Person
0,Fetterman's new chief of staff co-founded The ...,Trump-Russia,Moscow,Fetterman
1,@RyanPet70527777 @realpeterf @NEWSMAX Hes got ...,NEWSMAX-Hes,,"Donald-Trump,Trump"
2,@RVAwonk Listen to this 2017 interview by Ginn...,Trump-Intel,,Ginni-Thomas
3,@WhiteHouse when President Trump left office o...,WhiteHouse,,Trump
4,"""Elon Musk and Donald Trump are the two lone v...",,,"Elon-Musk,Donald-Trump"


### output  the file for visualisation

In [ ]:
# text_ner_out.to_csv('./processed_data/tweets_topics_sentiment_ner.csv', index=False)
text_ner_out = pd.read_csv('./processed_data/tweets_topics_sentiment_ner.csv')

In [ ]:
text_ner_out.loc[(text_ner_out['Place'] != '') | (text_ner_out['Organization'] != '')|(text_ner_out['Person'] != '')][['text','Organization','Place','Person']].head()


,text,Organization,Place,Person
0,Fetterman's new chief of staff co-founded The ...,Trump-Russia,Moscow,Fetterman
1,@RyanPet70527777 @realpeterf @NEWSMAX Hes got ...,NEWSMAX-Hes,NaN,"Donald-Trump,Trump"
2,@RVAwonk Listen to this 2017 interview by Ginn...,Trump-Intel,NaN,Ginni-Thomas
3,@WhiteHouse when President Trump left office o...,WhiteHouse,NaN,Trump
4,"""Elon Musk and Donald Trump are the two lone v...",NaN,NaN,"Elon-Musk,Donald-Trump"
